<h3>Importing Required Modules<h3/>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("../data/skylab_instagram_datathon_dataset.csv", sep=";")
df_original = df.copy(deep=True)
df.head()

<h3>Removing columns "period", "calculation_type" <h3\>

In [ ]:
#removing "period", "calculation_type" since the number of followers, likes,... are independent of these cathegories(Comment out after first run)
df = df_original.copy(deep=True) 
df.drop(columns=["period", "calculation_type", "domicile_country_name","primary_exchange_name", "compset", "legal_entity_name","ultimate_parent_legal_entity_name"], inplace=True)
#df.drop(columns=["period", "calculation_type", "domicile_country_name","primary_exchange_name", "compset"], inplace=True)

df.sort_values(by=["period_end_date"], inplace=True, ascending=False)
display(df.head())
display(df.shape)

In [ ]:
#Assumption: A brand will not be in different compset_groups over time!
df.drop_duplicates(inplace=True)

#now after removing duplicte rows, we can remove compset_group column
df.drop(columns=["compset_group"], inplace=True)

df.sort_values(by=["business_entity_doing_business_as_name", "period_end_date"], inplace=True, ascending=False)
df.reset_index(drop=True, inplace=True)
display(df.shape)
display(df.head())

In [ ]:
# #needed to evaluate "All Brands" brand data

# df_tests = df[df["business_entity_doing_business_as_name"] =="All Brands"]
# df_tests = df_tests[df_tests["compset_group"] =="Luxury & Premium & Mainstream"]
# df_tests.shape

# #get counts of unique values in each column
# for col in df_tests.columns:
#     print("Sum per column: ", df_tests[col].value_counts().sum())

#     print(col)
#     display(df_tests[col].value_counts())
#     print(len(df_tests[col].value_counts()))
#     print("\n")



In [ ]:
#df.pivot(index ="period_end_date", 
        #columns= ["compset_group", "business_entity_doing_business_as_name", "legal_entity_name", "ultimate_parent_legal_entity_name"],
        #values=["followers", "pictures", "videos", "comments", "likes"]).head()

#check for nan values in the dataset
nan_rows = df.isna().sum()
display(nan_rows)


#get rows with nan values
df[df.isna().any(axis=1)]


# columns = df.columns
# print(columns)
# df_sum = 0

# for i in columns:
    
#     print("\n")
#     print("-----------------------------------------------------------------------------")
#     print(i, "--> has length:", len(df[i].unique()))
#     df_i = df[i].value_counts().reset_index()
#     df_i.columns = [i, "count"]
#     display(df_i)
#     df_sum = df_i["count"].sum()
#     print("Sum of count:", df_sum)
#     df_sum = 0
#     print("-----------------------------------------------------------------------------")
#     print("\n")




<h3> Using Timedelta instead of Absolute datetime and ordering by datetime <h3>

In [ ]:
#taking timedelta from first available end date instead of absolute date (COMMENT OUT AFTER FIRST RUN)
#df['period_end_date'] = pd.to_datetime(df['period_end_date'])
#min_date = df['period_end_date'].min()
#df['period_end_date'] = df['period_end_date'] - min_date
#order by period_end_date
df.sort_values(by=["business_entity_doing_business_as_name", "period_end_date"], inplace=True, ascending=False)


<h3> where do we have missing values?<h3\>

In [ ]:
for name in df.columns:
    if df[df[name].isnull()].shape[0] > 0:
        print("number of missing values for ",name, ": ",df[df[name].isnull()].shape[0])

<h3>Investigation of Domicile Country Name<h3\>

In [ ]:
#COMMENT OUT AFTER FIRST RUN
#nan_domicile = df[df['domicile_country_name'].isnull()]
#nan_domicile["business_entity_doing_business_as_name"].unique().size
#check if brand has non-null domicile for each brand
#for brand in nan_domicile["business_entity_doing_business_as_name"].unique():
    #known_brands = []
    #check if brand has non-null domicile
    #if df[df["business_entity_doing_business_as_name"] == brand]["domicile_country_name"].notnull().any():
        #known_brands.append(brand)
        #print(brand)
#the result is empty, so we can't get the domicile from another row
#remove domicile_country_name column
#df.drop(columns=["domicile_country_name"], inplace=True)

In [ ]:
cathegories = ["followers", "likes", "comments", "videos", "pictures"]
for name in cathegories:
    print("number of missing values for ",name, ": ",df[df[name].isnull()].shape[0] + df[df[name] == 0].shape[0])
    print("number of zeros for ",name, ": ",df[df[name] == 0].shape[0])
    #ratio of zeros + nan to total
    print("ratio of zeros + nan to total for ",name, ": ",(df[df[name].isnull()].shape[0] + df[df[name] == 0].shape[0])/df.shape[0])
    


In [ ]:
#fill nan with zeros
for name in cathegories:
    df[name].fillna(0, inplace=True)

In [ ]:
# brands_missing_values = pd.DataFrame(columns=["brand", "cathegory", "ratio"])

# for name in df["business_entity_doing_business_as_name"].unique():
#     for cathegory in cathegories:
#         nr_missing_values = df[(df["business_entity_doing_business_as_name"] == name) & (df[cathegory].isnull() | (df[cathegory] == 0))].shape[0]
#         ratio = nr_missing_values / df[df["business_entity_doing_business_as_name"] == name].shape[0]
#         if ratio > 0.3:
#             brands_missing_values.loc[len(brands_missing_values.index)] = [name, cathegory, ratio]
#             #print("brand: ",name, " cathegory: ",cathegory, " ratio: ",ratio)
#brands_missing_values.to_csv("data/brands_missing_values.csv")

In [ ]:
#filling missing values with the mean of the brand 


# for name in df["business_entity_doing_business_as_name"].unique():
#     for cathegory in cathegories:
#         #if there are zeros or nan values for a brand, fill them with the mean of the brand
#         if df[(df["business_entity_doing_business_as_name"] == name) & (df[cathegory].isnull() | (df[cathegory] == 0))].shape[0] > 0:
#             mean = df[df["business_entity_doing_business_as_name"] == name][cathegory].mean()
#             df.loc[(df["business_entity_doing_business_as_name"] == name) & (df[cathegory].isnull() | (df[cathegory] == 0)), cathegory] = mean


In [ ]:
#df.pivot(index ="period_end_date", 
        #columns= ["compset_group", "business_entity_doing_business_as_name", "legal_entity_name", "ultimate_parent_legal_entity_name"],
        #values=["followers", "pictures", "videos", "comments", "likes"]).head()

#check for nan values in the dataset
nan_rows = df.isna().sum()
display(nan_rows)


#get rows with nan values
df[df.isna().any(axis=1)]


columns = df.columns
print(columns)
df_sum = 0

for i in columns:
    
    print("\n")
    print("-----------------------------------------------------------------------------")
    print(i, "--> has length:", len(df[i].unique()))
    df_i = df[i].value_counts().reset_index()
    df_i.columns = [i, "count"]
    display(df_i)
    df_sum = df_i["count"].sum()
    print("Sum of count:", df_sum)
    df_sum = 0
    print("-----------------------------------------------------------------------------")
    print("\n")




In [ ]:
display(df.head(10))
print(df.shape)

In [ ]:
display(df.tail(10))
print(df.shape)

In [ ]:
#save the cleaned data
df.to_csv("../data/cleaned_data.csv", index=False)